In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('fe_Chat_full.csv')
data.drop(columns=['No'], inplace=True)
data.drop('MVP', axis=1, inplace=True)
data.fillna('placeholder for answers', inplace=True)
data

,Questions,Answers
0,What happens to my data?,juli users collect a vast amount of accurate p...
1,What about data privacy?,Our mission at juli is to use data and artific...
2,Do you recommend a device to use with juli?,juli uses the data collected by your iPhone. T...
3,What is Heart Rate Variability?,Heart Rate Variability is how much the time be...
4,Why doesn't juli give me any recommendations h...,"Health is complicated, and so we need enough d..."
5,What could I do to improve my asthma?,Taking your preventive medication as directed ...
6,Is there anything I need to worry about today?,Most people with asthma should focus on pollen...
7,Should I see a doctor?,If you are having specific symptoms that indic...
8,What can I do to sleep better?,If your sleep problems are asthma-induced you ...
9,Can you tell me an asthma joke?,https://laffgaff.com/asthma-jokes-puns/


In [2]:
test_data = [
    "What is the meaning of juli name"]
#     "Can I speak to someone at juli?",
#     "Who is behind juli?",
#     "Who is juli"
# ]

In [3]:
# master fn for getting answers to questions

def genResults(questions, fn):
    def genresult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    result_df = pd.DataFrame(list(map(genresult, questions)), columns=['question', 'question_closest', 'answer', 'score'])
    return result_df['answer'].values, result_df['score'].values

In [4]:
def generate_results(question):
    answer, score, prediction = 'test','test','test'
    pass

## Bert QA system

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-mpnet-base-v2')

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 202kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 107kB/s]
Downloading: 100%|██████████| 10.1k/10.1k [00:00<00:00, 3.69MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 65.7kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 276kB/s] 
Downloading: 100%|██████████| 349/349 [00:00<00:00, 174kB/s]
Downloading: 100%|██████████| 438M/438M [00:33<00:00, 12.9MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 24.0kB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 127kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 667kB/s]
Downloading: 100%|██████████| 363/363 [00:00<00:00, 165kB/s]
Downloading: 100%|██████████| 13.1k/13.1k [00:00<00:00, 2.85MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 390kB/s]


In [7]:
def encode_questions():
    bc = model
    questions = data["Questions"].values.tolist()
    print("Questions count", len(questions))
    print("Start to calculate encoder....")
    questions_encoder = bc.encode(questions)
    np.save("question_emb", questions_encoder)
    questions_encoder_len = np.sqrt(
        np.sum(questions_encoder * questions_encoder, axis=1)
    )
    np.save("question_len_embedding", questions_encoder_len)
    print("Encoder ready")

encode_questions()

Questions count 57
Start to calculate encoder....
Encoder ready


In [24]:
class BertAnswer:
    def __init__(self):
        self.bc = model
        self.q_data = data["Questions"].values.tolist()
        self.a_data = data["Answers"].values.tolist()
        self.questions_encoder = np.load("question_emb.npy")
        self.questions_encoder_len = np.load("question_len_embedding.npy")

    def get(self, q):
        query_vector = self.bc.encode([q])[0]
        score = np.sum((query_vector * self.questions_encoder), axis=1) / (
            self.questions_encoder_len * (np.sum(query_vector * query_vector) ** 0.5)
        )
        top_id = np.argsort(score)[::-1][0]
        return self.a_data[top_id], score[top_id], self.q_data[top_id]

bm = BertAnswer()

def getBertAnswer(q):
    return bm.get(q)


# test_data = ['Who is behind juli?']
# test_data = ['Are there doctors in the Juli team?'] #0.8487
# test_data = ['Can we eat Juli?'] #0.5573
# test_data = ['Can we smoke during asthma?'] #0.5828
# test_data = ['Do you sell our data?'] # Is my data secure? 0.6194
test_data = ['Do you sell our data?']
# test_data = ['Who is behind juli?']


answer, score = genResults(test_data, getBertAnswer)
print(answer+ "--->with the score of: " +str(score) )

if score>0.65:
    print(answer)
else:
    print("Redirecting to the out of domain model")


['placeholder for answers--->with the score of: [0.61945295]']
Redirecting to the out of domain model
